In [28]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [29]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [30]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [31]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [32]:
dataset
dataset=dataset.replace({True: 1,False: 0})
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [33]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [34]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/5, random_state = 0)

In [35]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [36]:
from sklearn.svm import SVC

In [37]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [38]:
from sklearn.model_selection import GridSearchCV

param_grid = {'kernel':['linear','rbf','poly','sigmoid'],
             'gamma':['auto','scale'],
             'C':[10,100,1000,2000,3000]} 



grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 5,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=5)

In [39]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [40]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}: 0.950648360030511


In [41]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[55  3]
 [ 1 21]]


In [42]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96        58
           1       0.88      0.95      0.91        22

    accuracy                           0.95        80
   macro avg       0.93      0.95      0.94        80
weighted avg       0.95      0.95      0.95        80



In [43]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9670846394984326

In [44]:
table=pd.DataFrame.from_dict(re)

In [45]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.023862,1.009510e-02,0.021857,0.008471,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.796088,0.758892,0.740864,0.838040,0.887483,0.804273,0.053313,29
1,0.015624,3.504023e-07,0.003125,0.006251,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.875759,0.858747,0.875897,0.907389,0.968750,0.897308,0.039026,1
2,0.024996,7.653546e-03,0.003124,0.006248,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.827459,0.805648,0.793775,0.873807,0.919631,0.844064,0.046638,17
3,0.015625,2.193451e-06,0.000000,0.000000,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.733346,0.730321,0.718750,0.716066,0.825502,0.744797,0.040885,31
4,0.028127,6.250191e-03,0.003125,0.006250,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.796088,0.758892,0.740864,0.838040,0.887483,0.804273,0.053313,29
5,0.021876,7.655298e-03,0.003125,0.006250,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.875759,0.858747,0.875897,0.907389,0.968750,0.897308,0.039026,1
6,0.024999,7.654422e-03,0.006250,0.007655,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.827459,0.805648,0.793775,0.873807,0.919631,0.844064,0.046638,17
7,0.015625,1.868812e-06,0.000000,0.000000,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.733346,0.730321,0.718750,0.716066,0.825502,0.744797,0.040885,31
8,0.068747,7.654578e-03,0.003125,0.006250,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.796088,0.758892,0.740864,0.838040,0.920683,0.810913,0.064215,21
9,0.028120,6.252647e-03,0.009375,0.007655,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.858830,0.858747,0.891711,0.906923,0.968750,0.896992,0.040484,3


In [46]:
Age=float(input("Age:"))
EstimatedSalary=float(input("Estimated_Salary:"))
Gender_Male=int(input("Sex Male 1 or 0:"))

Age: 40
Estimated_Salary: 175000
Sex Male 1 or 0: 0


In [47]:
Future_Prediction=grid.predict([[Age,EstimatedSalary,Gender_Male]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


In [48]:
datasetcheck = pd.read_csv('Social_Network_Ads_ToBePredict.csv')

In [49]:
datasetcheck=pd.get_dummies(datasetcheck,drop_first=True)

In [50]:
datasetcheck

,User ID,Age,EstimatedSalary,Gender_Male
0,15624510,19,19000,True
1,15810944,35,20000,True
2,15668575,26,43000,False
3,15603246,27,57000,False
4,15804002,19,76000,True
...,...,...,...,...
395,15691863,46,41000,False
396,15706071,51,23000,True
397,15654296,50,20000,False
398,15755018,36,33000,True


In [51]:
datasetcheck=datasetcheck.replace({True: 1,False: 0})
datasetcheck

,User ID,Age,EstimatedSalary,Gender_Male
0,15624510,19,19000,1
1,15810944,35,20000,1
2,15668575,26,43000,0
3,15603246,27,57000,0
4,15804002,19,76000,1
...,...,...,...,...
395,15691863,46,41000,0
396,15706071,51,23000,1
397,15654296,50,20000,0
398,15755018,36,33000,1
